## Showing that finding a path in a Directed Graph can be gotten in Polynomial Time

In [3]:
from typing import List

def get_nodes(edges: List[List[str]]):
    nodes = []

    for edge in edges:

        if edge[0] not in nodes:
            nodes.append(edge[0])

        if edge[1] not in nodes:
            nodes.append(edge[1])
            
    nodes.sort()
    return nodes

## BRUTE-FORCE ALGORITHM

"""

"""

## EFFICIENT ALGORITHM

def checker(edges: List[List[str]], start: str, end: str) -> bool:
    
    marked = {start: True}

    for edge in edges: 
        if marked.get(edge[0]):
            marked[edge[1]] = True
            
    return True if marked.get(end) is not None else False


## Check that a path exists from a to f

edges = [["a", "b"], ["a", "c"], ["b", "e"], ["e", "c"], ["d", "f"]]
print(checker(edges, "a", "e"))

True


## Polynomial reduction function for proof that HAMPATH is NP-complete by showing that it can be reduced to a 3-SAT problem

## Proof of Cook Levin Theorem: SAT is NP-complete

The goal of this proof is to show that any abitrary NP computation can be converted to a satisfiability problem (SAT).

### Note

The Turing Machine(TM) implemented here is an Deterministic Turing Machine(DTM) instead of a Non-Deterministic Turing Machine (NDTM). We do this because a DTM is more intuitive to understand. 

However, this doesn't undermine the proof because any problem solvable by NDTM can be solved by DTM and you could also see the computation of the DTM as a path of a NDTM

### Steps

- The `DTM` takes in an arbitrary problem, computes it and returns the computations. This computation is 2-dimensional array of state of the TM at every step of the computation. This array is called a `tableau`
- Using the `Cook_Levin_Prover`, we convert each symbol in the array to a boolean variable set to true. And, then the we perform the following 4 checks on the variables.
    1. We check that every entry (i, j) in the computation has exactly one value. Therefore, we check that each entry has one or more values and that each entry cannot have 2 values.
    2. We check that the start state is in the first position of the first row `and` other symbols
    3. We check that an `ACCEPT_STATE` is in the tableau
    4. We check that each row yields the next one

In [57]:
from typing import List, Dict, Optional, Tuple, NewType
from enum import Enum, auto
from functools import reduce

BLANK = "BLANK"

ACCEPT_STATE = "ACCEPT_STATE"

class DIRECTION(Enum):
    LEFT = auto()
    RIGHT = auto()
    STAY = auto()
    
class STATE(Enum):
    ACCEPT = auto()
    REJECT = auto()
    RUNNING = auto()
    

Transition = NewType("Transition", Tuple[str, Optional[str], DIRECTION])

Transitions = NewType("Transitions", Dict[Tuple[str, str], Transition])

def pretty_print(computations):
    for step, config in enumerate(computations):
        print(f"Computation {step}: {config}")
        
class DTM:
    
    state = STATE.RUNNING
    
    head = []
            
    config: List[str] = []
    
    transitions: Transitions = {}
    
    computations: List[Dict] = []
    
    def __init__(self, start_state: str, start_config: List[str], transitions: Transitions):
        self.head = [start_state, 0]
        start_config.insert(0, start_state)
        self.config = start_config
        self.transitions = transitions
        self.computations = [start_config.copy()]
    
    @staticmethod
    def move(instruction: Tuple[Optional[str], str,  DIRECTION], config: List[str], head: List) -> Tuple[List[str], List]:
        config = config.copy()
        head = head.copy()
        index = head[1]
        (new_state, new_value, direction) = instruction
        if new_value == None:
            if direction == DIRECTION.LEFT:
                new_head_index = index - 1
                if new_head_index == -1:
                    config.insert(0, BLANK)
                    head[0] = new_state
                    head[1] = 0
                else:
                    head[0] = new_state
                    head[1] = new_head_index
            elif direction == DIRECTION.RIGHT:
                new_head_index = index + 1
                if new_head_index > len(config) - 1:
                    config.append(BLANK)
                    head[0] = new_state
                    head[1] = len(config) - 1
                else:
                    head[0] = new_state
                    head[1] = new_head_index

            else:
                pass
        else:
            config[head[1]] = new_value
            if direction == DIRECTION.LEFT:
                new_head_index = index - 1
                if new_head_index == -1:
                    config.insert(0, BLANK)
                    head[0] = new_state
                    head[1] = 0
                else:
                    head[0] = new_state
                    head[1] = new_head_index
            elif direction == DIRECTION.RIGHT:
                new_head_index = index + 1
                if new_head_index > len(config) - 1:
                    config.append(BLANK)
                    head[0] = new_state
                    head[1] = len(config) - 1
                else:
                    head[0] = new_state
                    head[1] = new_head_index
            else:
                pass
        return config, head
    
    @staticmethod
    def transition(transitions: Transitions, head: Tuple[str, int], config: List[str], computations: List[List[str]]):
        state = head[0]
        index = head[1]
        config.remove(state)
        symbol = config[index]
                                        
        move = transitions.get((state, symbol))
        
        if move == None:
            state = STATE.REJECT
            return state, head, config, computations
        elif move[0] == ACCEPT_STATE:
            state = STATE.ACCEPT
            new_state = move[0]
            new_value = move[1]
            direction = move[2]
            (config, head) = DTM.move((new_state, new_value, direction), config, head)
            state_index = head[1] - 1
            config.insert(state_index + 1, new_state)
            computations.append(config.copy())
            return state, head, config, computations
        
        new_state = move[0]
        new_value = move[1]
        direction = move[2]
        (config, head) = DTM.move((new_state, new_value, direction), config, head)
        state = STATE.RUNNING
        
        state_index = head[1] - 1
        config.insert(state_index + 1, new_state)
        computations.append(config.copy())
        return state, head, config, computations
                
            
    def run(self):
        (state, head, config, computations) = self.transition(self.transitions, self.head, self.config, self.computations)
        while state == STATE.RUNNING:
            (state, head, config, computations) = self.transition(self.transitions, head, config, computations)
        self.state = state
        self.head = head
        self.config = config
            
start_state = "q0"
start_config = ["0", "0", "0", "0"]
transitions = {
    ("q0", "0"): ("q1", BLANK, DIRECTION.RIGHT),
    ("q1", "x"): ("q1", None, DIRECTION.RIGHT),
    ("q1", "0"): ("q2", "x", DIRECTION.RIGHT),
    ("q2", "x"): ("q2", None, DIRECTION.RIGHT),
    ("q2", "0"): ("q3", None, DIRECTION.RIGHT),
    ("q3", "0"): ("q2", "x", DIRECTION.RIGHT),
    ("q2", BLANK): ("q4", None, DIRECTION.LEFT),
    ("q4", BLANK): ("q1", None, DIRECTION.RIGHT),
    ("q4", "x"): ("q4", None, DIRECTION.LEFT),
    ("q4", "0"): ("q4", None, DIRECTION.LEFT),
    ("q1", BLANK): (ACCEPT_STATE, None, DIRECTION.RIGHT),
}
tm = DTM(start_state, start_config, transitions)
tm.run()
pretty_print(tm.computations)

### Computation TO SAT

def bool_or(x: bool, y: bool):
    return x | y

def neg_bool_or(x: bool, y: bool):
    return (not x) | (not y)

def bool_and(x: bool, y: bool):
    return x & y

class Cook_Levin_Prover:
    
    def __init__(self, computations: List[List[str]], symbols: List[str], states: List[str]):
        self.computations = computations
        self.symbols = symbols
        self.states = states

    @staticmethod
    def computations_to_boolean_vars(computations: List[List[str]]) -> List[List[str]]:
        """
        Converts a set of computations to a booleans variables of true values
        """
        boolean_vars = []
        boolean_values = {}

        for i, computation in enumerate(computations):
            b_vars = [] 
            for j, symbol in enumerate(computation):
                name = f"x_{i}_{j}_{symbol}"
                b_vars.append(name)
                boolean_values[name] = True
            boolean_vars.append(b_vars)
        
        return boolean_vars, boolean_values
    
    @staticmethod
    def condition_1(symbols: List[str], boolean_vars: List[List[str]], boolean_values: Dict[str, bool]) -> bool:
        """
        We check that every entry (i, j) in the computation
        has exactly one value. Therefore, we check that each
        entry has one or more values and that each entry cannot
        have 2 values.
        """
        
        res = True
        for i, x in enumerate(boolean_vars):
            for j, y in enumerate(x):
                list_0 = []
                list_1 = []
                for s in symbols:
                    key_1 = f"x_{i}_{j}_{s}"
                    value_1 = (True if boolean_values.get(key_1) == True else False)
                    list_0.append(value_1)
                    for s in symbols:
                        key_2 = f"x_{i}_{j}_{s}"
                        if key_1 == key_2:
                            continue
                        value_2 = True if boolean_values.get(key_2) == True else False
                        list_1.append(neg_bool_or(value_1, value_2))
                        
                s_0 = reduce(bool_or, list_0)
                s_1 = reduce(bool_and, list_1)
                s_2 = bool_and(s_0, s_1)
                res = bool_and(res, s_2)
                
        return res
    
    @staticmethod
    def condition_2(boolean_vars: List[List[str]], boolean_values: Dict[str, bool]) -> bool:
        """
        We check that the start state is in the first position of the first row `and` other symbols
        """
        start_state = f"x_{0}_{0}_q0"
        first_row = [boolean_values.get(var) for var in boolean_vars[0]]
        return reduce(bool_and, first_row) and (start_state == boolean_vars[0][0])
    
    @staticmethod
    def condition_3(boolean_vars: List[List[str]], boolean_values: Dict[str, bool]) -> bool:
        """
        We check that an `ACCEPT_STATE` is in the tableau
        
        This is achieved by looking for an `ACCEPT_STATE` across all the cells(`boolean_vars[i][j]`)
        of the tableau and "ORing" them together.
        
        False is returned if not found.
        """
        accept_states = [boolean_values.get(var) for row in boolean_vars for var in row if ACCEPT_STATE in var]
        ## This serves as an identity element for when there are not accept states
        accept_states.append(False)
        return reduce(bool_or, accept_states)
    
    @staticmethod
    def condition_4(computations: List[List[str]], states: List[str]) -> bool:
        """
        We check that each row is a transition of the previous row
        """
        
        def get_state_and_index(computation: List[str], states: List[str]):
            for index, symbol in enumerate(computation):
                if symbol in states:
                    return index, symbol
                
        def get_2_by_3_window(state_index: int, current_row: List[str], next_row: List[str]):
            before = state_index - 1
            after = state_index + 1
            if before == -1:
                window = [[current_row[before], current_row[state_index], current_row[after]], [next_row[before], next_row[state_index], next_row[after]]]
            else:
                window = [[current_row[before], current_row[state_index], current_row[after]], [next_row[before], next_row[state_index], next_row[after]]]
                print(window)
        
        for row_index in range(len(computations)):
            if row_index == (len(computations) - 1):
                continue
            current_row = computations[row_index]
            next_row = computations[row_index + 1]
            state_in_current_row, index = get_state_and_index(current_row, states)
            window = get_2_by_3_window(state_in_current_row, current_row, next_row)
    
            
    def run(self):
        boolean_vars, boolean_values = self.computations_to_boolean_vars(self.computations)
        condition_1 = self.condition_1(self.symbols, boolean_vars, boolean_values)
        condition_2 = self.condition_2(boolean_vars, boolean_values)
        condition_3 = self.condition_3(boolean_vars, boolean_values)
        condition_4 = self.condition_4(self.computations, self.states)
        final_conditions = [condition_1, condition_2, condition_3]
        return reduce(bool_and, final_conditions)


symbols = ["q0", "q1", "q2", "q3", "q4", "0", "x", BLANK, ACCEPT_STATE]
states = ["q0", "q1", "q2", "q3", "q4", ACCEPT_STATE]
cook_levin_prover = Cook_Levin_Prover(tm.computations, symbols, states)
cook_levin_prover.run()

Computation 0: ['q0', '0', '0', '0', '0']
Computation 1: ['BLANK', 'q1', '0', '0', '0']
Computation 2: ['BLANK', 'x', 'q2', '0', '0']
Computation 3: ['BLANK', 'x', '0', 'q3', '0']
Computation 4: ['BLANK', 'x', '0', 'x', 'q2', 'BLANK']
Computation 5: ['BLANK', 'x', '0', 'q4', 'x', 'BLANK']
Computation 6: ['BLANK', 'x', 'q4', '0', 'x', 'BLANK']
Computation 7: ['BLANK', 'q4', 'x', '0', 'x', 'BLANK']
Computation 8: ['q4', 'BLANK', 'x', '0', 'x', 'BLANK']
Computation 9: ['BLANK', 'q1', 'x', '0', 'x', 'BLANK']
Computation 10: ['BLANK', 'x', 'q1', '0', 'x', 'BLANK']
Computation 11: ['BLANK', 'x', 'x', 'q2', 'x', 'BLANK']
Computation 12: ['BLANK', 'x', 'x', 'x', 'q2', 'BLANK']
Computation 13: ['BLANK', 'x', 'x', 'q4', 'x', 'BLANK']
Computation 14: ['BLANK', 'x', 'q4', 'x', 'x', 'BLANK']
Computation 15: ['BLANK', 'q4', 'x', 'x', 'x', 'BLANK']
Computation 16: ['q4', 'BLANK', 'x', 'x', 'x', 'BLANK']
Computation 17: ['BLANK', 'q1', 'x', 'x', 'x', 'BLANK']
Computation 18: ['BLANK', 'x', 'q1', 'x', 

True

## Application of `Schwartz-Zippel Lemma` in Graph Theory

In [40]:
## TODO

## The Sumcheck Protocol

In [145]:
from typing import List, Tuple
from math import log2


class FF:
    
    modulus: int
    
    def __init__(self, modulus: int):
        self.modulus = modulus
        
    def new(self, element: int) -> FFE:
        return FFE(element, self.modulus)
    
    def zero(self) -> FFE:
        return self.new(0)
    
    def one(self) -> FFE:
        return self.new(1)

class FFE:
    
    element: int
    
    modulus: int
    
    def __init__(self, element: int, modulus: int):
        self.element = element % modulus
        self.modulus = modulus
        
    def __add__(self, rhs: 'FFE'):
        if self.modulus != rhs.modulus:
            raise Exception("Modulus Mismatch")
        self.element = (self.element + rhs.element) % self.modulus
        return self
    
    def __sub__(self, rhs: 'FFE'):
        if self.modulus != rhs.modulus:
            raise Exception("Modulus Mismatch")
        self.element = (self.element - rhs.element) % self.modulus
        return self
    
    def __mul__(self, rhs: 'FFE'):
        if self.modulus != rhs.modulus:
            raise Exception("Modulus Mismatch")
        self.element = (self.element * rhs.element) % self.modulus
        return self


class Multilinear_Polynomial:
    
    evaluations: List[FFE]
    
    num_of_vars: int
    
    ff: FF
    
    def __init__(self, ff: FF, evaluations: List[FFE], num_of_vars: int):
        self.ff = ff
        num_of_evaluation_points = 2 ** num_of_vars
        length_of_evaluation_points = len(evaluations)
        if length_of_evaluation_points == num_of_evaluation_points:
            self.num_of_vars = num_of_vars
            self.evaluations = evaluations
        elif length_of_evaluation_points < num_of_evaluation_points:
            padding = [ff.zero() for _ in range(num_of_evaluation_points - length_of_evaluation_points)]
            evaluations = evaluations + padding
            self.evaluations = evaluations
            self.num_of_vars = num_of_vars
        else:
            self.evaluations = evaluations[:num_of_evaluation_points]
            self.num_of_vars = num_of_vars
            
            
    def evaluate(self, points: Tuple[FFE, int]) -> FFE:
        
        if len(points) != self.num_of_vars:
            raise Exception("Invalid number of points")
        
        res = self.ff.zero()
        
        for value in self.evaluations:
            res += value
            for 
            
        return res
        

ff = FF(17)
evaluations = [ff.new(2), ff.new(3), ff.new(4)]
mult_1 = Multilinear_Polynomial(ff, evaluations, 2)
mult_1.evaluate((ff.new(2), ff.new(2))).element

9

### Sumcheck in Matrix Multiplication `MatMult`